In [1]:
import pandas as pd
import numpy as np

# **Washing Machine/Data Cleaning**
1. ### Read Data
2. ### Clean Data
3. ### Save Data

### **Reading block** *(do either 1. or 2.)* 

1. ##### Single score file

In [9]:
subreddit = input('Name of subreddit (beware of capitalization): ')
scoreset = input('Filename of dataset containing scores: ')
scoreset_csv = scoreset + '.csv'
df = pd.read_csv(subreddit + '_dataset.csv')

scores = pd.read_csv(scoreset_csv).rename(columns={'id':'post_ID'}) # Renaming id column to match original dataset

Name of subreddit (beware of capitalization): science
Filename of dataset containing scores: science_set_scores


2. ##### Multiple score files

In [9]:
subreddit = input('Name of subreddit: ')
df = pd.read_csv(subreddit + '_dataset.csv')

score1 = pd.read_csv('dataset1 0-13334.csv').rename(columns={'id':'post_ID'}) # Renaming id column to match original dataset
score2 = pd.read_csv('dataset2 13334-26667.csv').rename(columns={'id':'post_ID'})
score3 = pd.read_csv('dataset3 26667-40100.csv').rename(columns={'id':'post_ID'})
scores = pd.concat([score1, score2, score3]) # Combining all score df's into one

### **Cleaning block**

In [11]:
df = pd.merge(df, scores, how='left', on='post_ID') #adding scraped scores to original dataset
df.drop('score_x', axis=1, inplace=True) #Removing old score column
df = df.rename(columns={'score_y':'score'}).dropna(subset=['score']) # Renaming score column & dropping rows where score == NaN
df.score = df.score.replace(r'[k]+$', '00', regex=True) # Reformatting score values from e.g. '19.2k' to '19.200'
df.score = df.score.replace(r'\.', '', regex=True).astype(int) # Reformatting score values from e.g. '19.200' to '19200' and changing datatype to 'int64'

df['title length'] = 0 #Generating title length column to dataset filled with 0s
df['time_hr'] = pd.DataFrame([x[11:] for x in df['timestamp']]) #Generating new column with only hours
df['24h_posttime'] = [0 for x in range(len(df))] #Generating new column filled with 0s

for x in range(len(df.title)): #Generating title length & 24hr_post_time columns
    df['title length'][x] = len(df['title'].iloc[x])
    for y in range(24):
        if int(df.time_hr.iloc[x][1]) == y:
            df['24h_posttime'].iloc[x] = y

df.drop('time_hr', axis=1, inplace=True) #Removing old timestamp

<ipython-input-11-c509463f8ba7>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['title length'][x] = len(df['title'].iloc[x])
C:\Users\wisev\.conda\envs\BDA\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


### **Saving block** *(change filename)*

In [12]:
df.to_csv('datasets\\' + subreddit + '_dataset_updated.csv', index=False)